## PLS

----

### 1. Import Modules

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using TensorFlow backend.


All packages successfully loaded


### 2. Load data and peak sheet

In [2]:
home = 'data/' 
file = 'ST000369.xlsx' 

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 163 TOTAL PEAKS: 180
Done!


### 3. Extract X & Y

In [3]:
peaklist = PeakTable['Name']
DataTable2 = DataTable[(DataTable['Class']==0) | (DataTable['Class']==1)]
Y = DataTable2['Class'].values 

### 4. Hyperparameters optimisation

In [4]:
# Set seed
seed = 37

# Split DataTable2 and Y into train and test (with stratification
DataTrain, DataTest, Ytrain, Ytest = train_test_split(DataTable2, Y, test_size=0.30, stratify=Y, random_state=seed)

# Extract and scale the metabolite data from the DataTable
peaklist = PeakTable['Name']                           
XT = DataTrain[peaklist]                                    
XTlog = np.log(XT)                                          
XTscale = cb.utils.scale(XTlog, method='auto')              
XTknn = cb.utils.knnimpute(XTscale, k=3)                                      
                                         
# param_dict
param_dict = {'n_components': [1,2,3,4,5,6]}                   
#param_dict = {'n_components': [1,2,3,4,5,6,7,8,9,10,11,12]} 


# Initalise
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                      
                                X=XTknn,                                 
                                Y=Ytrain,                               
                                param_dict=param_dict,                   
                                folds=5,
                                n_mc=100)                                

# Run and plot
cv.run()  
cv.plot(metric='r2q2', ci=95)

Number of cores set to: 8
Running ...


100%|██████████| 600/600 [00:07<00:00, 81.97it/s] 


Time taken: 0.13 minutes with 8 cores
Done!


Loading BokehJS ...

### 6. Get Model and Evaluate

In [5]:
# 6. Train, test and evaluate model

# Train model
model = cb.model.PLS_SIMPLS(n_components=2)
model.train(XTknn, Ytrain)

# Get X, Y
mu, sigma  = cb.utils.scale(XTlog, return_mu_sigma=True) 
peaklist = PeakTable.Name 
XV = DataTest[peaklist].values
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma)
XVknn = cb.utils.knnimpute(XVscale, k=3)
YVpred = model.test(XVknn)

# Evaluate Ypred against Ytest
evals = [Ytest, YVpred] 
model.evaluate(evals, cutoffscore=0.5, plot_median=False) 

print("AUC Train: {}".format(model.table['auc'][0][0]))
print("AUC Test: {}".format(model.table['auc'][1][0]))

Loading BokehJS ...

AUC Train: 0.96 (0.93, 0.98)
AUC Test: 0.89


In [6]:
# Get X
X = DataTable2[peaklist]                                    
Xlog = np.log(X)                                          
Xscale = cb.utils.scale(Xlog, method='auto')              
Xknn = cb.utils.knnimpute(Xscale, k=3)    

# Bootstrap evaluate
model.booteval(Xknn, Y, cutoffscore=0.5, bootnum=100) 

100%|██████████| 100/100 [00:00<00:00, 170.87it/s]


Loading BokehJS ...

### 7. Save table

In [7]:
home = 'tables/'
file = 'PLS_ST000369.xlsx'

model.save_table(home + file)

Done! Saved table as tables/PLS_ST000369.xlsx
